In [1]:
import subprocess
import time

In [2]:
dataset = "production"
bucket_method = "single"
cls_encoding = "laststate"

In [3]:
bucket_encoding = "agg"
cls_method = "xgboost"
results_dir = "results"

In [4]:
subprocess.Popen(("python experiments_param_optim_xgboost.py %s %s %s %s %s %s" % (dataset,
                                                                                   bucket_encoding,
                                                                                   bucket_method,
                                                                                   cls_encoding,
                                                                                   cls_method,
                                                                                   results_dir)).split())